In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import AgentEvent, ChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

In [3]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')
API_NINJAS_API_KEY=os.getenv('API_NINJAS_API_KEY')

In [4]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [5]:
model_client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

In [6]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
)

In [7]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- WebSearchAgent ----------
[FunctionCall(id='call_g15qBiPg1fVo1cmxJV8et93w', arguments='{"query": "Miami Heat player highest points 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_wTLjYyobCFhgWAqoqyQLKgpq', arguments='{"query": "Player total rebounds Miami Heat 2007-2008 season 2008-2009 season"}', name='search_web_tool')]
[Prompt tokens: 142, Completion tokens: 75]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n        ', call_id='call_g15qBiPg1fVo1cmxJV8et93w'), FunctionExecutionResult(content='The number of total rebounds for Dwayne 

Selector selected the previous speaker: WebSearchAgent


---------- WebSearchAgent ----------
The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, who scored 1,397 points.

As for the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, I would need the rebound data for the 2008-2009 season to determine that.
[Prompt tokens: 315, Completion tokens: 77]


Selector selected the previous speaker: WebSearchAgent


---------- WebSearchAgent ----------
[FunctionCall(id='call_Qog8hUKMtKgx8yc6RZVjlZev', arguments='{"query":"Dwyane Wade total rebounds Miami Heat 2008-2009 season"}', name='search_web_tool')]
[Prompt tokens: 399, Completion tokens: 29]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398.', call_id='call_Qog8hUKMtKgx8yc6RZVjlZev')]
---------- WebSearchAgent ----------
The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398.
---------- DataAnalystAgent ----------
[FunctionCall(id='call_nnWZ20gNP5LnK8gcVrYU9P2v', arguments='{"start":214,"end":398}', name='percentage_change_tool')]
[Prompt tokens: 333, Completion tokens: 20]
---------- DataAnalystAgent ----------
[FunctionExecutionResult(content='85.98130841121495', call_id='call_nnWZ20gNP5LnK8gcVrYU9P2v')]
---------- DataAnalystAgent ----------
85.98130841121495
---------- PlanningAgent ----------
1. Web

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=142, completion_tokens=75), content=[FunctionCall(id='call_g15qBiPg1fVo1cmxJV8et93w', arguments='{"query": "Miami Heat player highest points 2006-2007 season"}', name='search_web_tool'), FunctionCall(id='call_wTLjYyobCFhgWAqoqyQLKgpq', arguments='{"query": "Player total rebounds Miami Heat 2007-2008 season 2008-2009 season"}', name='search_web_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, content=[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 139

## Use Custom Selector Function

In [8]:
def selector_func(messages: Sequence[AgentEvent | ChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- PlanningAgent ----------
To address this query, we need to follow two main steps: identify the Miami Heat player with the highest points in the 2006-2007 season and calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons.

Here are the tasks broken down:

1. Web search agent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. Web search agent: Find the total rebounds for that player in the 2007-2008 and 2008-2009 NBA seasons.
3. Data analyst: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 seasons.
[Prompt tokens: 159, Completion tokens: 152]
---------- WebSearchAgent ----------
[FunctionCall(id='call_Kpdk51vPsWuLix9VSA2oqQop', arguments='{"que

Selector selected the previous speaker: WebSearchAgent


---------- WebSearchAgent ----------
[FunctionCall(id='call_RAeWK3VaMRa7yOly14ohuIuY', arguments='{"query": "Dwyane Wade total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_LI4GwC1ewaSip9T8kf4yYqCC', arguments='{"query": "Dwyane Wade total rebounds 2008-2009 season"}', name='search_web_tool')]
[Prompt tokens: 508, Completion tokens: 70]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.', call_id='call_RAeWK3VaMRa7yOly14ohuIuY'), FunctionExecutionResult(content='The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398.', call_id='call_LI4GwC1ewaSip9T8kf4yYqCC')]
---------- WebSearchAgent ----------
The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214.
The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398.
---------- PlanningAgent ----------
Now that we h

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=159, completion_tokens=152), content="To address this query, we need to follow two main steps: identify the Miami Heat player with the highest points in the 2006-2007 season and calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons.\n\nHere are the tasks broken down:\n\n1. Web search agent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. Web search agent: Find the total rebounds for that player in the 2007-2008 and 2008-2009 NBA seasons.\n3. Data analyst: Calculate the percentage change in the player's total rebounds between the 2007-2008 and 2008-2009 s